In [1]:
import ee
import datetime
import os
import itertools
import sys

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('../Process_1_GEE_Python_Classification/')

# import the temporal_consistency_check class
from BackGround_modules.Class_5_Temporal_consistency_check import Temporal_consistency_check

In [33]:
# define basic parameters
year_range = [f'{i}_{i+2}' for i in range(1990,2020,3)]

### Import Classified_random_img

In [35]:
# import the classified imgs
Classified_Landsat_1990_2019  = [ee.ImageCollection(f"users/wangjinzhulala/North_China_Plain_Python/classification_img/Control_{year}")
                                  .sum().gte(8).set('name',year)   for year in year_range]

Classified_Sentinel_2014_2019 = [ee.ImageCollection(f"users/wangjinzhulala/North_China_Plain_Python/classification_img/Sentinel_Landsat_{year}")
                                  .sum().gte(8).set('name',year)   for year in year_range[-2:]]

In [37]:
# Use the temporal_flag to indicate weather to use Sentinel classified imgs

temporal_flag = 'Landsat' #'Landsat'/'Sentinel'

# decide which classified img to be used
if temporal_flag == 'Landsat':
    Classified_imgs = Classified_Landsat_1990_2019
    
elif temporal_flag == 'Sentinel':
    Classified_imgs = Classified_Landsat_1990_2019[:-2] + Classified_Sentinel_2014_2019

else:
    print('Incorrect temporal_flag!')

In [38]:
Classified_imgs

In [56]:
Iter_temporal_check_instaces = Temporal_consistency_check(Classified_imgs,4,10).Iterate_the_check()

Check length is    ---> 4
Check weights is   ---> [4, 1, 1, 1]
Check threshold is ---> 6


In [61]:
Map = geemap.Map()
Map.setCenter(115.4508, 35.2492,10)

year_idx = 9

Map.add_basemap('HYBRID')
Map.addLayer(Classified_imgs[year_idx] ,{'min':0,'max':1},'origin')
Map.addLayer(Iter_temporal_check_instaces[0][year_idx] ,{'min':0,'max':1},'Iter_0')
Map.addLayer(Iter_temporal_check_instaces[1][year_idx] ,{'min':0,'max':1},'Iter_1')
Map.addLayer(Iter_temporal_check_instaces[2][year_idx] ,{'min':0,'max':1},'Iter_2')

Map

Map(center=[35.2492, 115.4508], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButt…